In [1]:
import os, ujson
from datetime import datetime, timedelta
from tchannel import thrift
from tchannel.sync import TChannel
from pyspark.sql.functions import *
import StringIO

## Postmaster service

In [2]:
os.chdir('/mnt/cephfs/hadoop-compute/phoenix/wangm/')

postmaster_service = thrift.load(path="postmaster.thrift", service="postmaster")

tchannel = TChannel(name="places")

## Query for response by places last week

In [3]:
current = datetime.now()
yesterday = current + timedelta(days=-1)
last_week = yesterday + timedelta(days=-7)

ugc_addr_query = """
    select
  d.rating.context as place_uuid,
  avg(
    CASE
      when d.rating.value like '%yes%' then 1
      else 0
    end
  ) as verify,
  avg(
    CASE
      when d.rating.value like '%no%' then 1
      else 0
    end
  ) as disagree,
  avg(
    CASE
      when d.rating.value like '%idk%' then 1
      else 0
    end
  ) as unverify,
  sum(
    CASE
      WHEN (d.rating.value like '%yes%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_verify,
  sum(
    CASE
      WHEN (d.rating.value like '%no%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_disagree,
  sum(
    CASE
      WHEN (d.rating.value like '%idk%'and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_unverify,
  count(d.rating.context) as total_vote,
  sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as total_score
from
  rawdata.schemaless_mezzanine_ratings_rows d
  left join MAP_CREATION.UGC_V0_SCORE_DSW_RIDER r on d.rating.reviewer_uuid = r.client_uuid
where
 array_contains(
    Array('address_yes', 'address_no', 'address_idk'),
    d.rating.value
  )
  and datestr > '2018-12-05'
  and datestr <= '{end_date}'
group by
  1
""".format(end_date = yesterday.strftime("%Y-%m-%d"))

print ugc_addr_query
ugc_addr_df = spark.sql(ugc_addr_query)
#refreshed_place_df.show()


    select
  d.rating.context as place_uuid,
  avg(
    CASE
      when d.rating.value like '%yes%' then 1
      else 0
    end
  ) as verify,
  avg(
    CASE
      when d.rating.value like '%no%' then 1
      else 0
    end
  ) as disagree,
  avg(
    CASE
      when d.rating.value like '%idk%' then 1
      else 0
    end
  ) as unverify,
  sum(
    CASE
      WHEN (d.rating.value like '%yes%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_verify,
  sum(
    CASE
      WHEN (d.rating.value like '%no%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_disagree,
  sum(
    CASE
      WHEN (d.rating.value like '%idk%'and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score

In [4]:
ugc_name_query = """
    select
  d.rating.context as place_uuid,
  avg(
    CASE
      when d.rating.value like '%yes%' then 1
      else 0
    end
  ) as verify,
  avg(
    CASE
      when d.rating.value like '%no%' then 1
      else 0
    end
  ) as disagree,
  avg(
    CASE
      when d.rating.value like '%idk%' then 1
      else 0
    end
  ) as unverify,
  sum(
    CASE
      WHEN (d.rating.value like '%yes%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_verify,
  sum(
    CASE
      WHEN (d.rating.value like '%no%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_disagree,
  sum(
    CASE
      WHEN (d.rating.value like '%idk%'and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_unverify,
  count(d.rating.context) as total_vote,
  sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as total_score
from
  rawdata.schemaless_mezzanine_ratings_rows d
  left join MAP_CREATION.UGC_V0_SCORE_DSW_RIDER r on d.rating.reviewer_uuid = r.client_uuid
where
  array_contains(
    Array('name_yes', 'name_no', 'name_idk'),
    d.rating.value
  )
  and datestr > '2018-12-05'
  and datestr <= '{end_date}'
group by
  1
""".format(end_date = yesterday.strftime("%Y-%m-%d"))

print ugc_name_query
ugc_name_df = spark.sql(ugc_name_query)
#refreshed_place_df.show()


    select
  d.rating.context as place_uuid,
  avg(
    CASE
      when d.rating.value like '%yes%' then 1
      else 0
    end
  ) as verify,
  avg(
    CASE
      when d.rating.value like '%no%' then 1
      else 0
    end
  ) as disagree,
  avg(
    CASE
      when d.rating.value like '%idk%' then 1
      else 0
    end
  ) as unverify,
  sum(
    CASE
      WHEN (d.rating.value like '%yes%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_verify,
  sum(
    CASE
      WHEN (d.rating.value like '%no%' and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score else 0 end) as score_disagree,
  sum(
    CASE
      WHEN (d.rating.value like '%idk%'and r.client_uuid is not null) THEN r.ugc_trust_score
      ELSE 0
    END
  ) / sum(case when r.client_uuid is not null then r.ugc_trust_score

## Total engaged rider

In [5]:
ugc_vote = """
 select
    DISTINCT
    d.rating.reviewer_uuid,
    d.rating.job_uuid
  from
    rawdata.schemaless_mezzanine_ratings_rows d
  where
    array_contains(
    Array('address_yes',
      'address_no',
      'address_idk',
      'name_no',
      'name_yes',
      'name_idk'),
    d.rating.value
  )
    and datestr > '{start_date}'
    and datestr <= '{end_date}'
""".format(start_date = last_week.strftime("%Y-%m-%d"), end_date = yesterday.strftime("%Y-%m-%d"))

print ugc_vote
ugc_vote_df = spark.sql(ugc_vote)


 select
    DISTINCT
    d.rating.reviewer_uuid,
    d.rating.job_uuid
  from
    rawdata.schemaless_mezzanine_ratings_rows d
  where
    array_contains(
    Array('address_yes',
      'address_no',
      'address_idk',
      'name_no',
      'name_yes',
      'name_idk'),
    d.rating.value
  )
    and datestr > '2019-01-10'
    and datestr <= '2019-01-17'



In [6]:
ugc_total_vote = """
 select
    DISTINCT
    d.rating.reviewer_uuid,
    d.rating.job_uuid
  from
    rawdata.schemaless_mezzanine_ratings_rows d
  where
    array_contains(
    Array('address_yes',
      'address_no',
      'address_idk',
      'name_no',
      'name_yes',
      'name_idk'),
    d.rating.value
  )
    and datestr > '2018-12-05'
    and datestr <= '{end_date}'
""".format(start_date = last_week.strftime("%Y-%m-%d"), end_date = yesterday.strftime("%Y-%m-%d"))

print ugc_total_vote
ugc_total_vote_df = spark.sql(ugc_total_vote)


 select
    DISTINCT
    d.rating.reviewer_uuid,
    d.rating.job_uuid
  from
    rawdata.schemaless_mezzanine_ratings_rows d
  where
    array_contains(
    Array('address_yes',
      'address_no',
      'address_idk',
      'name_no',
      'name_yes',
      'name_idk'),
    d.rating.value
  )
    and datestr > '2018-12-05'
    and datestr <= '2019-01-17'



In [7]:
# spark.version

## Total refreshed places

In [8]:
addr_df = ugc_addr_df.where(col("place_uuid").isNotNull())
addr_id = addr_df.select("place_uuid")

name_df = ugc_name_df.where(col("place_uuid").isNotNull())
name_id = name_df.select("place_uuid")

# X place address are voted as of xx-xx-xx
# addr_count = addr_id.count()

# X place name are voted as of xx-xx-xx
# name_count = name_id.count()

# X place are voted by rider as of xx-xx-xx
poi_count = addr_id.union(name_id).distinct().count()

In [9]:
addr_verified = addr_df.where(((col("score_verify") > 0.6) & (col("total_score") > 1)) | ((col("verify") > 0.6) & (col("total_vote") > 5)))

addr_disagree = addr_df.where(((col("score_disagree") > 0.6) & (col("total_score") > 1)) | ((col("disagree") > 0.6) & (col("total_vote") > 5)))

name_verified = name_df.where(((col("score_verify") > 0.6) & (col("total_score") > 1)) | ((col("verify") > 0.6) & (col("total_vote") > 5)))

name_disagree = name_df.where(((col("score_disagree") > 0.6) & (col("total_score") > 1)) | ((col("disagree") > 0.6) & (col("total_vote") > 5)))

# X good place address are verified as of xx-xx-xx
addr_verified_count = addr_verified.count()

# X bad place address are identified as of xx-xx-xx
addr_disagree_count = addr_disagree.count()

# X good place name are verified as of xx-xx-xx
name_verified_count = name_verified.count()

# X bad place name are identified as of xx-xx-xx
name_disagree_count = name_disagree.count()

# X riders submitted Y votes in the past week
ugc_vote_count = ugc_vote_df.count()
ugc_rider_count = ugc_vote_df.select("reviewer_uuid").distinct().count()

# X riders submitted Y votes as of xx-xx-xx
ugc_total_vote_count = ugc_total_vote_df.count()
ugc_total_rider_count = ugc_total_vote_df.select("reviewer_uuid").distinct().count()

print ugc_rider_count, 'riders submitted', ugc_vote_count, 'votes in the past week'
print poi_count, ' place are voted by riders as of now'
print addr_verified_count, 'good poi addresses are verified'
print name_verified_count, 'good poi names are verified'
print addr_disagree_count, 'bad poi addresses are identified'
print name_disagree_count, 'bad poi names are identified'

81373 riders submitted 82980 votes in the past week
199557  place are voted by riders as of now
40435 good poi addresses are verified
52765 good poi names are verified
1437 bad poi addresses are identified
945 bad poi names are identified


# Email notification

In [10]:
ugc_email_content = StringIO.StringIO()
today_text = current.strftime('%Y-%m-%d')

top_text = '''
    <!DOCTYPE html>
    <html>
        <head>
            <meta charset="utf-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Places UGC of Week from {week_start} to {week_end}</title>
            <meta name="description" content="Places UGC Weekly Newletter">
        </head>
        <body>
            <em>This is an autogenerated email on Places UGC for week from {week_start} to {week_end}</em>
            
            <div>
                <h3>Weekly UGC response </h3>
                <div>Weekly engaged riders: <b>{ugc_rider_count}</b></div>
                <div>Weekly response submitted: <b>{ugc_vote_count}</b></div>
            </div>

            <div>
                <h3>Total UGC response </h3>
                <div>Total engaged riders: <b>{ugc_total_rider_count}</b></div>
                <div>Total response submitted: <b>{ugc_total_vote_count}</b></div>
            </div>

            <div>
                <h3>Total UGC response breakdown by POI attribute and response type</h3>
'''.format(week_start=last_week.strftime("%Y-%m-%d"),
           week_end=yesterday.strftime("%Y-%m-%d"), 
           ugc_rider_count=ugc_rider_count,
           ugc_vote_count=ugc_vote_count,
           ugc_total_rider_count=ugc_total_rider_count,
           ugc_total_vote_count=ugc_total_vote_count
           )
ugc_email_content.write(top_text)

ugc_email_content.write('''
                <div>
                    <div>Total voted POI: <b>{poi_count}</b></div>
                    <div>Rider verified Good POI addresses: <b>{addr_verified}</b></div>
                    <div>Rider verified Good POI names: <b>{name_verified}</b></div>
                    <div>Rider reported Bad POI address: <b>{addr_disagree}</b></div>
                    <div>Rider reported bad POI names: <b>{name_disagree}</b></div>
                </div>
                '''.format(poi_count=poi_count,
                           addr_verified=addr_verified_count,
                           name_verified=name_verified_count,
                           addr_disagree=addr_disagree_count,
                           name_disagree=name_disagree_count
                          )
)

# bottom_text = '''
#             <div>
#                 <h3>Provider Refresh this week</h3>
#                 <div>
#                     <div>Provider refresh count: <b>{}</b></div>
#                 </div>
#                 <div>
#                     <div>Name diff count: <b>{}</b></div>
#                 </div>
#                 <div>
#                     <div>Address diff count: <b>{}</b></div>
#                 </div>
#                 <div>
#                     <div>Status diff count: <b>{}</b></div>
#                 </div>
#             </div>    
#         </body>
#     </html>    
# '''.format(provider_refresh_count, name_diff_count, address_diff_count, status_diff_count)
# refresh_email_content.write(bottom_text)

In [11]:
subject_arg = yesterday.strftime('Living Maps - Rider UGC for Places Week of %Y-%m-%d')

rawEmail_struct = postmaster_service.RawEmail(subject = subject_arg, richBody = ugc_email_content.getvalue())
content_arg = postmaster_service.Content(rawEmail = rawEmail_struct)

# to_arg = postmaster_service.Recipient(emailAddress = 'wangm@uber.com')

to_arg = postmaster_service.Recipient(emailAddress = 'maps-places-mce@uber.com')

cc_arg = [postmaster_service.Recipient(emailAddress= 'niravm@uber.com')]

recipients_struct = postmaster_service.Recipients(to = to_arg, cc = cc_arg)

fromEmail_arg = 'noreply@uber.com'
messageType_arg = 'internal'
request_struct = postmaster_service.Request(fromEmail = fromEmail_arg, recipients = recipients_struct, content = content_arg, messageType=messageType_arg)

future = tchannel.thrift(postmaster_service.Postmaster.testEmail(request = request_struct))
print future.result(timeout = 30000).body

Result({'providerPayload': u'MIME-Version: 1.0\r\nContent-Transfer-Encoding: quoted-printable\r\nContent-Type: text/html; charset=utf-8\r\nFrom: <noreply@uber.com>\r\nTo: <maps-places-mce@uber.com>\r\nSubject: =?UTF-8?Q?Living_Maps_-_Rider_UGC_for_Places_Week_of_2019-01-17?=\r\nX-Uber-Id: 970e758e-d24b-4bee-906a-259838c5e1bb\r\nX-Smtpapi: {"filters":{"bypass_list_management":{"settings":{"enable":1}},"clicktrack":{"settings":{"enable":1}}},"unique_args":{"from_email":"noreply@uber.com","message_uuid":"970e758e-d24b-4bee-906a-259838c5e1bb","unique_id":"970e758e-d24b-4bee-906a-259838c5e1bb"}}\r\n\r\n\r\n    <!DOCTYPE html>\r\n    <html>\r\n        <head>\r\n            <meta charset=3D"utf-8">\r\n            <meta name=3D"viewport" content=3D"width=3Ddevice-width, initia=\r\nl-scale=3D1.0">\r\n            <title>Places UGC of Week from 2019-01-10 to 2019-01-17</title>\r\n            <meta name=3D"description" content=3D"Places UGC Weekly Newlett=\r\ner">\r\n        </head>\r\n        <bo